## Fine tune Resnet-18
This code is for fine-tuning the ResNet model on the DermaMnist dataset.

The DermaMnist dataset is a collection of dermatological images used for skin disease classification. 
Fine-tuning involves taking a pre-trained ResNet model and adapting it to the specific task of classifying images in the DermaMnist dataset.

The process typically includes:
- Loading the pre-trained ResNet model.
- Modifying the final layers to match the number of classes in the DermaMnist dataset.
- Training the modified model on the DermaMnist dataset.
- Evaluating the performance of the fine-tuned model.

This approach leverages the pre-trained features of the ResNet model, which can lead to better performance and faster convergence compared to training a model from scratch.


In [1]:
# import torch
# import torchvision.models as models
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Dataset
import numpy as np
# Load the pre-trained ResNet-18 model
resnet18 = models.resnet18(pretrained=True)

# Print the model architecture
print(resnet18.fc)

c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Linear(in_features=512, out_features=1000, bias=True)


In [2]:
class DermaMNISTDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image = self.dataframe['image'][idx]  # (28, 28, 3) numpy array
        # image = (image * 255).astype(np.uint8)  # Convert to uint8 for transforms
        label = self.dataframe['label'][idx]
        
        # Convert numpy image to PIL Image for applying transforms
        image = transforms.ToPILImage()(image)
        image = transforms.ToTensor()(image)
        if self.transform:
            image = self.transform(image)
        
        # print(type(image))
        return image, label


In [3]:
%reload_ext kedro.ipython

[11/06/24 22:02:33] INFO     Using                                                                  ]8;id=697339;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=626517;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\framework\project\__init__.py#249\249]8;;\
                             'c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\framewor                
                             k\project\rich_logging.yml' as logging configuration.                                 

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=930105;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=806267;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py#58\58]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=273697;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=973249;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py#60\60]8;;\

                    INFO     Resolved project path as: c:\Users\Admin\Desktop\onco-derm-ai.         ]8;id=801354;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=440932;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py#175\175]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[11/06/24 22:02:38] INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=4791;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=26160;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py#62\62]8;;\

...

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=790492;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro_mlflow\config\kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=736718;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro_mlflow\config\kedro_mlflow_config.py#260\260]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///C:/Users/Admin/Desktop/onco-derm-ai/mlruns'                                  

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=841981;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=709844;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro_telemetry\plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[11/06/24 22:02:39] INFO     Kedro project onco-derm-ai                                             ]8;id=906601;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=462446;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=644004;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=869787;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py#142\142]8;;\
                             'pipelines'                                                                           

[11/06/24 22:02:41] INFO     Registered line magic 'run_viz'                                        ]8;id=68636;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=113889;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\ipython\__init__.py#148\148]8;;\

In [4]:
catalog


{'train_intermediate': 'kedro.io.memory_dataset.MemoryDataset()',
 'train_raw': "kedro_datasets.pickle.pickle_dataset.PickleDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/onco-derm-ai/data/01_raw/train.pkl'), "
              "backend='pickle', protocol='file', load_args={}, save_args={})",
 'val_raw': "kedro_datasets.pickle.pickle_dataset.PickleDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/onco-derm-ai/data/01_raw/val.pkl'), "
            "backend='pickle', protocol='file', load_args={}, save_args={})",
 'test_raw': "kedro_datasets.pickle.pickle_dataset.PickleDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/onco-derm-ai/data/01_raw/test.pkl'), "
             "backend='pickle', protocol='file', load_args={}, save_args={})",
 'pre-processed_train_data': "kedro_datasets.pickle.pickle_dataset.PickleDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/onco-derm-ai/data/02_intermediate/pre-processed_train_data.pkl'), "
                             "backend='pickle',

In [5]:
train_data = catalog.load("pre-processed_train_data")

[11/06/24 22:06:10] INFO     Loading data from pre-processed_train_data (PickleDataset)...      ]8;id=222371;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=858859;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

In [ ]:
train_data['image'][0]

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),     # Resize to 224x224
    # transforms.ToTensor(),             # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load your dataset
train_dataset = DermaMNISTDataset(train_data, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize ResNet-18 model
model = models.resnet18(pretrained=True)
num_classes = 7
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Adjust final layer for DermaMNIST classes

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_epochs = 10  # Adjust as needed

train_loader

[11/06/24 22:06:15] WARNING  c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\torchvision\mod ]8;id=114520;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\warnings.py\warnings.py]8;;\:]8;id=490193;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\warnings.py#109\109]8;;\
                             els\_utils.py:208: UserWarning: The parameter 'pretrained' is                         
                             deprecated since 0.13 and may be removed in the future, please use                    
                             'weights' instead.                                                                    
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\torchvision\mod ]8;id=484540;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\warnings.py\warnings.py]8;;\:]8;id=944425;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\warnings.py#109\109]8;;\
                             els\_utils.py:223: UserWarning: Arguments other than a weight enum or                 
                             `None` for 'weights' are deprecated since 0.13 and may be removed in                  
                             the future. The current behavior is equivalent to passing                             
                             `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use                            
                             `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.                
                               warnings.warn(msg)                                                                  
                                                                                                                   

In [7]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        print(type(images))
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        # print(labels)
        labels = labels.squeeze().long()
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
    # Save the model checkpoint
torch.save(model.state_dict(), 'resnet18_dermamnist.pth')
print("Pre-training complete.")

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'tor

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:16                                                                                   │
│                                                                                                  │
│   13 │   │                                                                                       │
│   14 │   │   # Backward and optimize                                                             │
│   15 │   │   optimizer.zero_grad()                                                               │
│ ❱ 16 │   │   loss.backward()                                                                     │
│   17 │   │   optimizer.step()                                                                    │
│   18 │   │                                                                                       │
│   19 │   │   running_loss += loss.item()                                                         │
│                                                                                                  │
│ c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\torch\_tensor.py:581 in backward          │
│                                                                                                  │
│ c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\torch\autograd\__init__.py:347 in         │
│ backward                                                                                         │
│                                                                                                  │
│ c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\torch\autograd\graph.py:825 in            │
│ _engine_run_backward                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
# Ensure labels are in the correct shape and data type


# Forward pass
outputs = model(images)
loss = criterion(outputs, labels)